In [1]:
import sys, os
os.getcwd()

'/home/lohrasp/lohrasp/analyticsoptim/stock_screener'

# Get database connection

In [116]:
from sqlalchemy import create_engine
SQLALCHEMY_DATABASE_URL = f"postgresql://{params['user']}:{params['password']}@{params['host']}/{params['database']}"
engine = create_engine(SQLALCHEMY_DATABASE_URL)
all.to_sql("security",con=engine,schema="saffron", index=False,if_exists='append')

# Adding the symbol to table

In [2]:
import pandas as pd
nasdaq = pd.read_csv ('/home/lohrasp/lohrasp/Robinhood_RR/AllStocks/NASDAQ.csv')
nyse = pd.read_csv ('/home/lohrasp/lohrasp/Robinhood_RR/AllStocks/NYSE.csv')
amex = pd.read_csv ('/home/lohrasp/lohrasp/Robinhood_RR/AllStocks/AMEX.csv')
rating_100 = pd.read_csv ('/home/lohrasp/lohrasp/Robinhood_RR/AllStocks/rating_100m.csv')
rating_10b = pd.read_csv ('/home/lohrasp/lohrasp/Robinhood_RR/AllStocks/rating_10b.csv')



In [35]:
all = pd.concat([nasdaq,nyse,amex])

In [37]:
all = all.rename({"Symbol":"symbol", "Name":"name","MarketCap":"marketcap", "IPOyear":"ipoyear",
                 "Sector":"sector" } ,axis =1)

In [48]:
all =all.drop("LastSale", axis =1)

In [92]:
m = "nan"
def get_money(m):
#     print(m, type(m))
    if not isinstance(m,str):
        return m
    if isinstance(m,str):
        if (m is None) or (m.lower()=='nan'):
            return m
    abb = m[-1]
    m = float (m[1:-1])
    if abb == 'B':
        m = m*1000
    return m
for mm in all.marketcap:
    get_money(mm)

In [114]:
all.ipoyear = all.ipoyear.fillna(0).astype(int)

In [94]:
all["marketcap"] = all.marketcap.apply(get_money)

In [7]:
allsymbols = set(nyse.Symbol).union(amex.Symbol).union (nasdaq.Symbol)

In [9]:
len(allsymbols)

6978

In [23]:
import psycopg2 as db


In [21]:
def config(filename='../frontend/database.ini', section='postgresql'):

    # create a parser
    print(os.getcwd(),filename)
    assert os.path.isfile(filename)
    parser = ConfigParser()
    # read config file
    parser.read(filename)
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db
from configparser import  ConfigParser


In [125]:
params = config(filename="../frontend/database.ini")
conn = db.connect(**params)
c = conn.cursor()
symboltocheck = "BSSS"
c.execute(f"""SELECT * FROM saffron."security" where symbol='{symboltocheck}'""")
c.fetchmany(10)

/home/lohrasp/lohrasp/analyticsoptim/stock_screener ../frontend/database.ini


[]

In [122]:
!pwd

/home/lohrasp/lohrasp/analyticsoptim/stock_screener


# Mark s&p 500

In [128]:
df = pd.read_csv("sp500.csv", header=None)

In [136]:
for s in df[0]:
    c.execute(f"""UPDATE saffron."security" SET sp500='true' where symbol='{s}'""")
    conn.commit()


In [137]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get stock info
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 163000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, and OneDrive; LinkedIn that includes Talent, Learning, Sales, and Marketing solutions, as well as premium subscriptions; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. Its Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consu

In [140]:
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 163000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, and OneDrive; LinkedIn that includes Talent, Learning, Sales, and Marketing solutions, as well as premium subscriptions; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. Its Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consu

c.close()
conn.close()